# Requirements

In [ ]:
!pip install -r requirements.txt

# Imports

In [ ]:
from scrapers.iloveqatar import ILoveQatarScraper
from scrapers.visitqatar import VisitQatarScraper
from models import Event
from typing import List
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Initaliaze scrapers and functions

In [ ]:
# Initialize scrapers
scrapers = [
    ILoveQatarScraper(),
    VisitQatarScraper()
]

In [ ]:
def run_scrapers(scrapers: list) -> List[Event]:
    all_events = []
    for scraper in scrapers:
        try:
            print(f"\n{'='*50}")
            print(f"Running {scraper.source_name} scraper...")
            events = scraper.scrape_events()
            all_events.extend(events)
            print(f"Found {len(events)} events from {scraper.source_name}")
            
            # Save individual scraper results
            scraper.save_to_csv(events)
        except Exception as e:
            print(f"Error with {scraper.source_name} scraper: {e}")
    
    return all_events

# Run 

In [ ]:
all_events = run_scrapers(scrapers)

In [ ]:
events_df = pd.DataFrame([event.to_dict() for event in all_events])
events_df.head()

# Statistics

In [ ]:
print("\nEvent Statistics:")
print(f"Total events: {len(all_events)}")
# By source
source_counts = events_df['source'].value_counts()
print("\nBy source:")
print(source_counts)
# By category (if available)
if 'category' in events_df.columns:
    category_counts = events_df['category'].value_counts()
    print("\nBy category:")
    print(category_counts)

In [ ]:
plt.figure(figsize=(10, 5))
source_counts.plot(kind='bar', color=['skyblue', 'lightgreen'])
plt.title('Number of Events by Source')
plt.xlabel('Source')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

In [ ]:
if 'category' in events_df.columns:
    plt.figure(figsize=(12, 6))
    category_counts.plot(kind='bar', color='lightcoral')
    plt.title('Number of Events by Category')
    plt.xlabel('Category')
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# Save results

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"combined_events_{timestamp}.csv"
events_df.to_csv(filename, index=False)
print(f"\nSaved {len(all_events)} events to {filename}")

# Raw inspection

In [ ]:
print("\nSample Event Details:")
for idx, event in enumerate(all_events[:3], 1):  # Show first 3 events
    print(f"\nEvent {idx}:")
    print(f"Title: {event.title}")
    print(f"Date: {event.start_date} to {event.end_date}")
    print(f"Time: {event.time}")
    print(f"Location: {event.location}")
    print(f"Source: {event.source}")